# Trabalho final

Disciplina: Aprendizado de Máquina

Aluno: Carlos Monteiro


Replicação e melhorias do artigo ""

https://ieeexplore.ieee.org/document/10673571 -> Não foi implementado por vazar dados uma vez que criou um atributo de preço por pé quadrado. Ele implementa o XG-Boost e avalia com RMSE e R2 (tbm com acuracia, mas fala no próprio texto que é a mesma coisa).

https://ieeexplore.ieee.org/document/10543370 -> Neste artigo, o autor usou mais dados de imóveis da mesma área além do conjunto usado aqui.

https://ieeexplore.ieee.org/document/9074952

Imports necessários

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import kagglehub
import subprocess
import os

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import xgboost as xgb

from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.neural_network import MLPRegressor

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score

# 1. Importar dados

In [2]:
if not os.path.exists("kangglehub_datasets"):  
    path = kagglehub.dataset_download("amitabhajoy/bengaluru-house-price-data")
    subprocess.run(["cp", "-r", path, "./kangglehub_datasets/"])
df = pd.read_csv('kangglehub_datasets/Bengaluru_House_Data.csv')
df

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00
...,...,...,...,...,...,...,...,...,...
13315,Built-up Area,Ready To Move,Whitefield,5 Bedroom,ArsiaEx,3453,4.0,0.0,231.00
13316,Super built-up Area,Ready To Move,Richards Town,4 BHK,NaN,3600,5.0,NaN,400.00
13317,Built-up Area,Ready To Move,Raja Rajeshwari Nagar,2 BHK,Mahla T,1141,2.0,1.0,60.00
13318,Super built-up Area,18-Jun,Padmanabhanagar,4 BHK,SollyCl,4689,4.0,1.0,488.00


# 2. Replicação do Artigo
https://ieeexplore.ieee.org/document/9362864

## 2.1 Pré processamento

In [3]:
print(df.isnull().sum())

area_type          0
availability       0
location           1
size              16
society         5502
total_sqft         0
bath              73
balcony          609
price              0
dtype: int64


Retirar a coluna area_type e society (ela será retirada por ser uma coluna categórica que tem 5502 valores faltantes dos 13320)

In [4]:
df.drop(['area_type', 'society'], axis=1, inplace=True)
df

,availability,location,size,total_sqft,bath,balcony,price
0,19-Dec,Electronic City Phase II,2 BHK,1056,2.0,1.0,39.07
1,Ready To Move,Chikka Tirupathi,4 Bedroom,2600,5.0,3.0,120.00
2,Ready To Move,Uttarahalli,3 BHK,1440,2.0,3.0,62.00
3,Ready To Move,Lingadheeranahalli,3 BHK,1521,3.0,1.0,95.00
4,Ready To Move,Kothanur,2 BHK,1200,2.0,1.0,51.00
...,...,...,...,...,...,...,...
13315,Ready To Move,Whitefield,5 Bedroom,3453,4.0,0.0,231.00
13316,Ready To Move,Richards Town,4 BHK,3600,5.0,NaN,400.00
13317,Ready To Move,Raja Rajeshwari Nagar,2 BHK,1141,2.0,1.0,60.00
13318,18-Jun,Padmanabhanagar,4 BHK,4689,4.0,1.0,488.00


Ajusta as colunas necessárias

In [5]:
df['availability'] = [1 if x.strip() == ('Ready To Move' or 'Immediate Possession')
                       else 0 for x in df['availability']]

In [6]:
df['size'] = df['size'].str.extract(r'(\d+)').astype('float16')
df['size'] = df['size'].fillna(df['size'].mean())

In [7]:
df['total_sqft'] = df['total_sqft'].str.extract(r'(\d+)').astype('float16')

In [8]:
df['bath'] = df['bath'].fillna(df['bath'].mean())

In [9]:
df['balcony'] = df['balcony'].fillna(df['balcony'].mean())

In [10]:
df.dropna(subset=['location'], inplace=True)
df.reset_index(drop=True, inplace=True)

Converter as colunas não numéricas em One Hot Encoder

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13319 entries, 0 to 13318
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   availability  13319 non-null  int64  
 1   location      13319 non-null  object 
 2   size          13319 non-null  float16
 3   total_sqft    13319 non-null  float16
 4   bath          13319 non-null  float64
 5   balcony       13319 non-null  float64
 6   price         13319 non-null  float64
dtypes: float16(2), float64(3), int64(1), object(1)
memory usage: 572.4+ KB


In [12]:
encoder = OneHotEncoder(sparse_output=False) 
encoded = encoder.fit_transform(df['location'].values.reshape(-1, 1))
df_encoded = pd.DataFrame(
    encoded, 
    columns=encoder.get_feature_names_out(['location'])
)
df = pd.concat([df, df_encoded], axis=1)
df.drop(columns=['location'], inplace=True)
df

,availability,size,total_sqft,bath,balcony,price,location_ Anekal,location_ Banaswadi,location_ Basavangudi,location_ Bhoganhalli,...,location_rr nagar,location_sankeswari,location_sapthagiri Layout,location_sarjapura main road,location_singapura paradise,location_t.c palya,location_tc.palya,location_vinayakanagar,"location_white field,kadugodi",location_whitefiled
0,0,2.0,1056.0,2.0,1.000000,39.07,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,4.0,2600.0,5.0,3.000000,120.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,3.0,1440.0,2.0,3.000000,62.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,3.0,1521.0,3.0,1.000000,95.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,2.0,1200.0,2.0,1.000000,51.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13314,1,5.0,3452.0,4.0,0.000000,231.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13315,1,4.0,3600.0,5.0,1.584376,400.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13316,1,2.0,1141.0,2.0,1.000000,60.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13317,0,4.0,4688.0,4.0,1.000000,488.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Separa o conjunto de dados em treinamento e teste

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns=['price']), df['price'], test_size=0.2, random_state=42)

## 2.2 Aplicação dos Algoritmos

In [15]:
model_list = [DecisionTreeRegressor(random_state=42), 
              RandomForestRegressor(random_state=42), 
              SVR(), 
              xgb.XGBRegressor(random_state=42)]
model_name_list = ['Decision Tree', 'Random Forest', 'SVR', 'XGBoost']
r2_list = []
rmse_list = []
mse_list = []
mae_list = []
for model in model_list:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    r2_list.append(model.score(X_test, y_test))
    rmse_list.append(np.sqrt(mean_squared_error(y_test, y_pred)))
    mse_list.append(mean_squared_error(y_test, y_pred))
    mae_list.append(mean_absolute_error(y_test, y_pred))

## 2.3 Resultados

Resultados do Artigo

In [16]:
r2_paper_list = [0.29, 0.61, 0.31, 0.63]
rsme_paper_list = [119.56, 88.68, 118.14, 85.69]
mse_paper_list = [14294.38, 7864.09, 13956.22, 7342.85]
mae_paper_list = [39.67, 32.09, 41.29, 32.30]

Na tabela abaixo são apresentadas as métricas para cada modelo adquirida na replicação em comparação com o valor apresentado no artigo.

In [28]:
print(f'Model          |     R2     |     RMSE      |       MSE         |   MAE   ')
print(f'---------------|------------|---------------|-------------------|-----------')

for i, modelo in enumerate(model_name_list):
    print(f'{modelo.ljust(15)}| {r2_list[i]:1.2f}/{r2_paper_list[i]:1.2f}  | {rmse_list[i]:6.2f}/{rsme_paper_list[i]:6.2f} | {mse_list[i]:8.2f}/{mse_paper_list[i]:8.2f} | {mae_list[i]:5.2f}/{mae_paper_list[i]:5.2f} ')

Model          |     R2     |     RMSE      |       MSE         |   MAE   
---------------|------------|---------------|-------------------|-----------
Decision Tree  | 0.46/0.29  | 112.50/119.56 | 12656.60/14294.38 | 35.89/39.67 
Random Forest  | 0.63/0.61  |  93.51/ 88.68 |  8744.17/ 7864.09 | 30.97/32.09 
SVR            | 0.30/0.31  | 127.68/118.14 | 16301.68/13956.22 | 43.13/41.29 
XGBoost        | 0.57/0.63  |  99.87/ 85.69 |  9973.13/ 7342.85 | 34.42/32.30 


# 3. Aplicação dos Base Line

In [14]:
model2_list = [LinearRegression(), 
              Lasso(alpha=0.1), #Regularização L1
              Ridge(alpha=1.0), #Regularização L2
              MLPRegressor(hidden_layer_sizes=[10, 10], activation='relu', solver='lbfgs', alpha=1e-3, max_iter=5000)] 
model2_name_list = ['Linear Regression', 'Lasso Regression', 'Ridge Regression', 'MLP Regressor']
mse_base_list = []
mae_base_list = []
for model in model2_list:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse_base_list.append(mean_squared_error(y_test, y_pred))
    mae_base_list.append(mean_absolute_error(y_test, y_pred))

In [15]:
print(f'Model            |    MSE   |   MAE   ')
print(f'-----------------|----------|---------')

for i, modelo in enumerate(model2_name_list):
    print(f'{modelo.ljust(17)}| {mse_base_list[i]:7.2f} | {mae_base_list[i]:5.2f} ')

Model            |    MSE   |   MAE   
-----------------|----------|---------
Linear Regression| 11295406864742359040.00 | 156450746.43 
Lasso Regression | 12522.06 | 43.59 
Ridge Regression | 11543.98 | 41.05 
MLP Regressor    | 14967.33 | 54.58 


Model            |    MSE   |   MAE   
-----------------|----------|---------
Linear Regression| 10756.65 | 44.69 
Lasso Regression | 10753.85 | 44.68 
Ridge Regression | 10756.51 | 44.69 
MLP Regressor    | 10735.12 | 45.74

Por que a Regreção Linear apresenta erros tão grandes quando se utiliza a localização em OHE?

**Próximos passos:**

Fazer otimização de Hiperparâmetros com Gried Shearch ou Interact

Observar a importancia dos atributos